In [46]:
from pymatgen.io.vasp.inputs import Kpoints
from pymatgen.core import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.symmetry.kpath import KPathLatimerMunro, KPathBase, KPathSetyawanCurtarolo, KPathSeek
from pymatgen.symmetry.bandstructure import HighSymmKpath
from pymatgen.io.vasp.inputs import Poscar
import os
import subprocess as sp
import numpy as np
import ase.io as io
from ase.spacegroup import get_spacegroup

In [11]:
def get_kpoint_brillouin_zone(path, dest, mocha, rel_type):
    struct = Structure.from_file(f"{path}scf/CONTCAR")
    # struct = Structure.from_file(f"{path}POSCAR")
    spg_analy =SpacegroupAnalyzer(struct)
    prim_struct=spg_analy.get_primitive_standard_structure(international_monoclinic=False) # based off of Curtarolo convention
    pos = Poscar(prim_struct)
    if not os.path.exists(f"{path}/band_curtarolo"):
        os.mkdir(f"{path}/band_curtarolo")
    # if not os.path.exists(f"{path}/scf_band"):
    #     os.mkdir(f"{path}/scf_band")
    # pos.write_file(f"{path}band/POSCAR_PRIMITIVE.vasp")
    # pos.write_file(f"{path}scf_band/POSCAR_PRIMITIVE.vasp")

    x = input("\nChoose high-symmetry k-path convention (methods can be found on https://pymatgen.org/pymatgen.symmetry.html#module-pymatgen.symmetry.kpath):\n\nl: Latimer-Munro\ns: Setyawan-Curtarolo\nh: Hinuma\n\nYour option: ")
    if x == 'l':
        kpath = KPathLatimerMunro(prim_struct)
        meth = "LatiMunro"
    elif x == 's':
        # kpath = KPathSetyawanCurtarolo(prim_struct)
        kpath = HighSymmKpath(prim_struct) #this is the same as the setyawan thing
        meth = "SetyaCurta"
    elif x == 'h':
        kpath = KPathSeek(prim_struct)
        meth = "Hinuma"
    # kpath = HighSymmKpath(prim_struct)
    else:
        print("Heathen! You did not choose a method! I will choose Latimer-Munro for you.\n\n")
        kpath = KPathLatimerMunro(prim_struct)

    kpts = Kpoints.automatic_linemode(divisions=10,ibz=kpath)
    kpts.write_file(f"{dest}k-point_brillouin_zone/{mocha}_{rel_type}_{meth}")
    kpts.write_file(f"{path}band_curtarolo/KPOINTS")
    print(f"KPOINTS written to {path}band_curtarolo/KPOINTS")
    print(f"Primitive structure written to {path}band_curtarolo/POSCAR_PRIMITIVE.vasp")

In [55]:
mocha_list = ['AgS_2-methyl_ester_1D_Lum_YELLOW', 
              'AgS_2-butane_GREEN', 'AgS_3-methoxy_YELLOW', 'AgS_2-methoxy_1D_Lum_GREEN',
              'AgSe_Py_RED', 'Au_SEtPh_low-temp_RED', 'Au_SPh-4F_RED']    

# mocha_list = ["galac_rtr_dehydrated"]

# root = f"/Users/adrianaladera/Desktop/MIT/research/mochas/VASP_calculations/sugar_mochas/"
root = "/Users/adrianaladera/Desktop/MIT/research/mochas/VASP_calculations/fluorescent_1D"
destination = "/Users/adrianaladera/Desktop/MIT/research/CSE_thesis/VASP_shit/"

for mocha in mocha_list:
    if os.path.isdir(f"{root}/{mocha}/H-only_relax/"):
        print(mocha)
        get_kpoint_brillouin_zone(f"{root}/{mocha}//H-only_relax/", destination, mocha, "2ISIF")
        # command = f"cd {root}/{mocha}/band && cp INCAR POSCAR POTCAR run.slurm ../band_curtarolo && cd ../band_curtarolo && cp ../scf_band/CHG* ."
        # sp.run(command, shell=True)

galac_rtr_dehydrated
KPOINTS written to /Users/adrianaladera/Desktop/MIT/research/mochas/VASP_calculations/sugar_mochas//galac_rtr_dehydrated//H-only_relax/band_curtarolo/KPOINTS
Primitive structure written to /Users/adrianaladera/Desktop/MIT/research/mochas/VASP_calculations/sugar_mochas//galac_rtr_dehydrated//H-only_relax/band_curtarolo/POSCAR_PRIMITIVE.vasp


In [63]:
ass = True
root = "/Users/adrianaladera/Desktop/MIT/research/mochas/VASP_calculations/fluorescent_1D/AgS_1-naphthyl_YELLOW/"

# structure = io.read(f"{root}scf/CONTCAR")
# sp = get_spacegroup(structure,symprec=1e-5)
# print(sp)

if ass == True:
    struct = Structure.from_file(f"{root}scf/CONTCAR")
    # struct = Structure.from_file(f"{path}POSCAR")
    spg_analy = SpacegroupAnalyzer(struct, symprec=0.1)
    system = spg_analy.get_crystal_system()
    space_group = struct.get_space_group_info()
    lattice_type = spg_analy.get_lattice_type()
    spg_symbol = spg_analy.get_space_group_symbol()
    lattice_params = struct.lattice.parameters
    print(system, space_group, lattice_type, spg_symbol)
    print(lattice_params)

    prim_struct=spg_analy.get_primitive_standard_structure(international_monoclinic=False) # based off of Curtarolo convention
    spg_analy = SpacegroupAnalyzer(prim_struct)
    system = spg_analy.get_crystal_system()
    space_group = prim_struct.get_space_group_info()
    lattice_type = spg_analy.get_lattice_type()
    spg_symbol = spg_analy.get_space_group_symbol()
    lattice_params = prim_struct.lattice.parameters
    print(system, space_group, lattice_type, spg_symbol)
    print(lattice_params)

    conv = SpacegroupAnalyzer(prim_struct).get_conventional_standard_structure(international_monoclinic=False)
    a,b,c =conv.lattice.abc
    alpha = conv.lattice.alpha


    pos = Poscar(prim_struct)
    # if not os.path.exists(f"{root}/band_curtarolo"):
    #     os.mkdir(f"{root}/band_curtarolo")
    # if not os.path.exists(f"{path}/scf_band"):
    #     os.mkdir(f"{path}/scf_band")
    # pos.write_file(f"{path}band/POSCAR_PRIMITIVE.vasp")

    kpath = KPathSetyawanCurtarolo(prim_struct).mcl(b, c, alpha * np.pi / 180)
    # kpath = KPathSetyawanCurtarolo(prim_struct).orc()

    lattice_params = struct.lattice.parameters
    lattice_params_new = prim_struct.lattice.parameters

    # print()
    # print(lattice_type, lattice_params)

    kpoints = []
    labels = []
    for path in kpath["path"]:
        kpoints.append(kpath["kpoints"][path[0]])
        labels.append(path[0])
        for i in range(1, len(path) - 1):
            kpoints += [kpath["kpoints"][path[i]]] * 2
            labels += [path[i]] * 2

        kpoints.append(kpath["kpoints"][path[-1]])
        labels.append(path[-1])

    kpts = Kpoints(
            "Line_mode KPOINTS file",
            style=Kpoints.supported_modes.Line_mode,
            coord_type="Reciprocal",
            kpts=kpoints,
            labels=labels,
            num_kpts=10,
        )
    # print(kpts)
    
    # path = "/Users/adrianaladera/Desktop/MIT/research/mochas/VASP_calculations/fluorescent_1D/AgS_1-naphthyl_YELLOW/"
    # kpts.write_file(f"{destination}k-point_brillouin_zone/{mocha}_2ISIF_SetyaCurta")
    # kpts.write_file(f"{root}band_curtarolo/KPOINTS")
    # print(f"KPOINTS written to {root}band_curtarolo/KPOINTS")
    # print(f"Primitive structure written to {root}band_curtarolo/POSCAR_PRIMITIVE.vasp")


triclinic ('P1', 1) triclinic P1
(16.186000824, 4.4310002327, 24.198999404861002, 90.0, 97.13999938957228, 90.0)
triclinic ('P1', 1) triclinic P1
(4.4310002327, 16.186000824, 24.198999404861002, 97.13999938957228, 90.0, 90.0)
